In [12]:
import os
import pandas as pd
import re

In [13]:
def parse_lua_file(filename):
    with open(filename, 'r') as file:
        content = file.read()

    # Use regular expressions to find all variables and their values
    pattern = r'(local |bundled_variables\.)beta_(\w+) = ([-.\d]+)'
    matches = re.findall(pattern, content)

    # Get model name from file name (without .lua extension)
    model_name = os.path.splitext(os.path.basename(filename))[0]

    # Return a list of dictionaries, each representing a row in the final DataFrame
    result = []
    for match in matches:
        try:
            initial = float(match[2])
        except ValueError:
            initial = 0    # None  # or np.nan, or 0, or any value we want to use for invalid floats
        result.append({
            'model': model_name,
            'param': f'{match[0]}beta_{match[1]} = {match[2]}',
            'module': 'preday',
            'include': False,
            'initial': initial
        })
    return result

# Full path to 'folder_lua'
folder_path = "/home/s210911/simmobility/scripts/lua/mid/behavior_vc"

lua_files = [f for f in os.listdir(folder_path) if f.endswith('.lua')]

# Sort the files in lexicographical (alphabetical) order before processing
lua_files.sort()

# Use folder_path when opening the file
rows = [row for lua_file in lua_files for row in parse_lua_file(os.path.join(folder_path, lua_file))]


# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(rows)


df=df[pd.to_numeric(df['initial'], errors='coerce').notnull()]
df["provided_initial"]=df["initial"].copy()

# Adding cons_travel 

# New data to add as a row
new_row_data = {
    'model': 'dpb',
    'param': 'local cons_travel = 1.85',
    'module': 'preday',
    'include': False,
    'initial': 1.85,
    'provided_initial': 1.85
}

# Append the new row to the DataFrame
df = df.append(new_row_data, ignore_index=True)

print(df)

# # Create a DataFrame from the list of dictionaries
# df = pd.DataFrame(rows)
# df['initial'] = pd.to_numeric(df['initial'], errors='coerce')
# df["provided_initial"] = df["initial"].copy()
# df['initial'] = df['initial'].fillna(0)
# print(df)


     model                                        param  module  include  \
0      dpb                 local beta_homemaker = 0.442  preday    False   
1      dpb                  local beta_retired = -0.419  preday    False   
2      dpb                   local beta_fulltime = 4.10  preday    False   
3      dpb                   local beta_parttime = 3.02  preday    False   
4      dpb               local beta_selfemployed = 2.60  preday    False   
...    ...                                          ...     ...      ...   
1856  wdmw      local beta_age65_zero_car_motor = 0.844  preday    False   
1857  wdmw  local beta_age65_one_plus_car_motor = -1.24  preday    False   
1858   zpa                   local beta_size = -0.99053  preday    False   
1859   zpa                        local beta_MRTd = 1.6  preday    False   
1860   dpb                     local cons_travel = 1.85  preday    False   

      initial  provided_initial  
0     0.44200           0.44200  
1    -0.41900      

In [14]:
# Chosen model parameters to calibrate
models_to_include = ['dpb', 'imd', 'nte', 'tmdo']
df.loc[df['model'].isin(models_to_include), 'include'] = True

In [19]:
# Save the DataFrame to a csv file
#df.to_csv('parameters.csv', index=False)
